### 1. Dynamic Graph CNN base model.

##### Imports

In [2]:
import os
import sys
import copy
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

##### Model Implementation for Semantic Segmentation
Starting with <strong>PointNet</strong> implementation, as it will be our backbone, on top of which we will add the EdgeConv

In [ ]:
class PointNet(nn.Module):
    def __init__(self, args, out_channels=40):
        super(PointNet, self).__init__()
        self.conv_1 = nn.Conv1d(3, 64, kernel_size=1, bias=False)
        self.conv_2 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv_3 = nn.Conv1d(64, 64, kernel_size=1, bias=False)
        self.conv_4 = nn.Conv1d(64, 128, kernel_size=1, bias=False)
        self.conv_5 = nn.Conv1d(128, args.embedded_dims, kernel_size=1, bias=False)
        self.batch_norm_1 = nn.BatchNorm1d(64)
        self.batch_norm_2 = nn.BatchNorm1d(64)
        self.batch_norm_3 = nn.BatchNorm1d(64)
        self.batch_norm_4 = nn.BatchNorm1d(128)
        self.batch_norm_5 = nn.BatchNorm1d(args.embedded_dims)
        self.fc_1 = nn.Linear(args.embedded_dims, 512, bias=False)
        self.batch_norm_6 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout()
        self.fc_2 = nn.Linear(512, out_channels)
    
    def forward(self, x):
        x = F.relu(self.batch_norm_1(self.conv_1(x)))
        x = F.relu(self.batch_norm_2(self.conv_2(x)))
        x = F.relu(self.batch_norm_3(self.conv_3(x)))
        x = F.relu(self.batch_norm_4(self.conv_4(x)))
        x = F.relu(self.batch_norm_5(self.conv_5(x)))
        x = F.adaptive_max_pool1d(x, 1).squeeze()
        x = F.relu(self.batch_norm_6(self.fc_1(x)))
        x = self.dropout(x)
        x = self.fc_2(x)
        return x